## Identifying and resolving conflicts in a batch of VI files

Andes Reinspection 
LRG
Data information can be found at https://data.desi.lbl.gov/desi/spectro/prospect/target-reinspection-andes/

/global/cfs/cdirs/desi/spectro/redux/andes


448 visual inspections of a total of 150 unique objects

32 conflicts

In [79]:
save_filename='truth_table_Andes_reinspection_LRG_68001_20200315_3.csv'

In [80]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

import VI_utils as VI
sys.path.append("/global/homes/r/tlan/prospect/prospect/py")

from prospect import utils_specviewer,plotframes
#import desispec
# library location will change ..
sys.path.append("/global/u2/t/tlan/python/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [81]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/'
tiles = ['68001']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
#petals = ['0','3','6','7']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
#del obs_db['67230-20200314']
#obs_db.keys()

In [82]:
pd.set_option('display.max_rows', 10)

In [83]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/projects/VI_files/Andes_reinspection/LRG/'

In [84]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(7,10):
    pattern = "desi*_"+str(68001)+"_"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

print(vi_files)

desi*_68001_7_*.csv
desi*_68001_8_*.csv
desi*_68001_9_*.csv
['desi-vi_LRG_reinspection_68001_7_KSD.csv', 'desi-vi_LRG_reinspection_68001_7_PNT.csv', 'desi-vi_LRG_reinspection_68001_8_KSD.csv', 'desi-vi_LRG_reinspection_68001_8_PNT.csv', 'desi-vi_LRG_reinspection_68001_9_KSD.csv', 'desi-vi_LRG_reinspection_68001_9_PNT.csv']


In [85]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = ",", engine='python')
print(vi_files[0])
for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = ",", engine='python')
    vi = vi.append(vi2, ignore_index=True)
#print(vi)    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TARGETID'])

print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_LRG_reinspection_68001_7_KSD.csv
desi-vi_LRG_reinspection_68001_7_PNT.csv
desi-vi_LRG_reinspection_68001_8_KSD.csv
desi-vi_LRG_reinspection_68001_8_PNT.csv
desi-vi_LRG_reinspection_68001_9_KSD.csv
desi-vi_LRG_reinspection_68001_9_PNT.csv
There are 300 visual inspections of a total of 150 unique objects
That gives 2.0 inspections per object.


In [86]:
#vi is a dataframe
#display(vi)

# Merge with zbest files
#Add: fiberID, delta_chi2, flux information,.. anything else?


In [87]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')

for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

#tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_panda()    
EXPID = list(set(tf['EXPID']))[0]
tf = tf[tf['EXPID']==EXPID]
tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()
print(len(tspec_df))
print(len(tf_df))
#tf_df = tf_df.rename(columns={"TargetID": "TARGETID"})
#tspec_df = tspec_df.rename(columns={"TargetID": "TARGETID"})
#print(len(vi))
vi = vi.merge(tf_df, how='left', on='TARGETID')
vi = vi.merge(tspec_df, how='left', on='TARGETID')
print(len(vi))

5000
5000
300


In [88]:
print(vi.keys())
#print(len(vi))
#vi['best redshift']=vi['VI_z']

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')


### Adding a bunch of useful columns

In [89]:
#vi['VI_z']
import VI_utils as VI
#vi['VI_z']
#bad_str = [s is None for s in vi['VI_comment']]
#vi['VI_comment'].isnull()
#bad_str = [s is None for s in vi['VI_comment']]
#print(bad_str)
#vi.loc[vi['VI_comment'].isnull(),'VI_comment']='--'
#vi['VI_comment']
#vi['all VI comments'] = vi.groupby('TARGETID')['VI_comment'].transform(lambda x: '|'.join(x))
#print(vi['all VI comments'])
vi = VI.prep_new_columns(vi)
#vi['best redshift'] = vi['VI_z']
#test = vi['VI_z']
#vi['best redshift'].isnull()
#vi.loc[vi['best redshift'].isnull(), 'best redshift'] = vi.loc[vi['best redshift'].isnull(), 'Redrock_z']
#vi['best redshift']

In [90]:
#check all the new columns (keys) have been added correctly
display(vi)
#print(vi.keys())

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35191277557384124,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.0534,KSD,...,134.008479,1.0534,GALAXY,3.5,1,0.000000e+00,1.0534,--|--,2,none
1,35191277557385194,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.7980,KSD,...,154.027289,0.7980,GALAXY,3.5,1,6.174767e-17,0.7980,--|--,2,none
2,35191277561580179,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.9798,KSD,...,179.828303,0.9798,GALAXY,3.5,1,0.000000e+00,0.9798,--|--,2,none
3,35191277565773296,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.6643,KSD,...,6.163075,0.5143,GALAXY,2.5,1,7.594268e-01,1.6643,possibly a blend; funny continuum|Continuum re...,2,none
4,35191277565773729,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.2419,KSD,...,345.014871,1.2419,GALAXY,3.5,1,0.000000e+00,1.2419,--|--,2,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,35191281181263036,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.1632,NaN,...,127.712819,1.1632,GALAXY,4.0,0,0.000000e+00,1.1632,--|--,2,none
296,35191281181263268,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.7820,NaN,...,83.020963,0.7820,GALAXY,4.0,0,0.000000e+00,0.7820,--|--,2,none
297,35191281181264139,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.9746,NaN,...,88.901476,0.9746,GALAXY,2.0,2,1.359374e-01,0.9746,--|Continuum is resolved But Very noisy signal...,2,none
298,35191281181264649,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.8672,NaN,...,163.699856,0.8672,GALAXY,4.0,0,0.000000e+00,0.8672,--|--,2,none


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [91]:
vi_gp = vi.groupby(['TARGETID'])
vi_conflict = VI.find_conflicts(vi_gp)

In [92]:
len(vi_gp)

150

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [93]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35191274000614197 35191274004808525 35191277565773296 35191277569966650
 35191277590937828 35191277590937990 35191277590938241 35191277590938713
 35191277595133531 35191277599329412 35191277603523851 35191281143513341
 35191281147710346 35191281160292265 35191281177068074 35191281181264139]
Total number of conflicts to resolve:  16


In [94]:
unique_targets[0]

35191274000614197

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [95]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TARGETID==unique_targets[conflict_id]][['TARGETID', 
       'Redrock_spectype', 'Redrock_z', 'VI_scanner', 'VI_quality', 'VI_issue',
       'VI_z', 'VI_spectype', 'VI_comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
    # VI interface in notebook
    if show_spectra:
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [96]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [97]:
show_spectra = False

In [98]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
38,35191274000614197,GALAXY,0.3792,KSD,3,S,0.4424,NaN,poor sky subraction around 7000 AA; superposit...,0.4424,GALAXY,3.5,1,0.3792,0.045824,poor sky subraction around 7000 AA; superposit...,2,none
88,35191274000614197,GALAXY,0.3792,PNT,4,S,0.3792,GALAXY,At around lambda=7000A there is a mismatch due...,0.3792,GALAXY,3.5,1,0.3792,0.045824,poor sky subraction around 7000 AA; superposit...,2,none


In [99]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.4424
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad sky subtraction. Redshif is robust.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

38    4
88    4
Name: VI_quality, dtype: int64
conflict ID =  0


In [100]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
42,35191274004808525,GALAXY,0.965,KSD,2,NaN,0.965,NaN,possible blend w/z=0.9198,0.965,GALAXY,3.0,2,0.965,0.0,possible blend w/z=0.9198|--,2,none
92,35191274004808525,GALAXY,0.965,PNT,4,NaN,0.965,GALAXY,--,0.965,GALAXY,3.0,2,0.965,0.0,possible blend w/z=0.9198|--,2,none


In [101]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.8101
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] =3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'relative low S/N CaII, but should be robust.'
##look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)



conflict ID =  1


In [102]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
3,35191277565773296,GALAXY,1.6643,KSD,3,NaN,0.5143,NaN,possibly a blend; funny continuum,0.5143,GALAXY,2.5,1,1.6643,0.759427,possibly a blend; funny continuum|Continuum re...,2,none
53,35191277565773296,GALAXY,1.6643,PNT,2,NaN,1.6643,GALAXY,Continuum resolved No feature is deteced,1.6643,GALAXY,2.5,1,1.6643,0.759427,possibly a blend; funny continuum|Continuum re...,2,none


In [103]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.5143
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. Redshift is robust. CaII + NaD'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  2


In [104]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
9,35191277569966650,QSO,0.817,KSD,3,NaN,0.8191,NaN,odd template around OIII,0.8191,QSO,3.5,1,0.81805,0.001156,odd template around OIII|--,2,none
59,35191277569966650,QSO,0.817,PNT,4,NaN,0.8170,GALAXY,--,0.8170,GALAXY,3.5,1,0.81805,0.001156,odd template around OIII|--,2,none


In [105]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.8191
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY' 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift is robust.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)


conflict ID =  3


In [106]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
120,35191277590937828,GALAXY,0.6315,KSD,3,R,0.6369,NaN,spurious signal modeled as negative OIII - coi...,0.6369,GALAXY,3.5,1,0.6315,0.00331,spurious signal modeled as negative OIII - coi...,2,none
170,35191277590937828,GALAXY,0.6315,PNT,4,NaN,0.6315,GALAXY,Ca(K) missing from the model,0.6315,GALAXY,3.5,1,0.6315,0.00331,spurious signal modeled as negative OIII - coi...,2,none


In [107]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.6360
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Very bad calibration.'
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

conflict ID =  4


In [108]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
121,35191277590937990,GALAXY,0.5668,KSD,0,NaN,0.5668,NaN,Two objects in spectrum,0.5668,GALAXY,2.0,4,0.5678,0.001276,Two objects in spectrum|Calibration of the red...,2,none
171,35191277590937990,GALAXY,0.5668,PNT,4,R,0.5688,GALAXY,Calibration of the redshift using Ca(K) and Ca...,0.5688,GALAXY,2.0,4,0.5678,0.001276,Two objects in spectrum|Calibration of the red...,2,none


In [109]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 0
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Two objects in the spectrum'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  5


In [110]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
123,35191277590938241,GALAXY,1.0662,KSD,2,R,1.0662,NaN,negative OII in model,1.0662,GALAXY,3.0,2,1.0662,0.0,negative OII in model|--,2,none
173,35191277590938241,GALAXY,1.0662,PNT,4,NaN,1.0662,GALAXY,--,1.0662,GALAXY,3.0,2,1.0662,0.0,negative OII in model|--,2,none


In [111]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.5
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'CaII should be robust.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  6


In [112]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
125,35191277590938713,QSO,0.0767,KSD,2,NaN,0.7532,NaN,--,0.7532,QSO,3.0,2,0.0767,0.613325,--|--,2,none
175,35191277590938713,QSO,0.0767,PNT,4,C,0.0867,GALAXY,--,0.0867,GALAXY,3.0,2,0.0767,0.613325,--|--,2,none


In [113]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.7532
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Reshift should be secure'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  7


In [114]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
128,35191277595133531,GALAXY,0.7773,KSD,2,NaN,0.7773,NaN,--,0.7773,GALAXY,3.0,2,0.7773,0.0,--|--,2,none
178,35191277595133531,GALAXY,0.7773,PNT,4,NaN,0.7773,GALAXY,--,0.7773,GALAXY,3.0,2,0.7773,0.0,--|--,2,none


In [115]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1920
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = "Redshift is secure."

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  8


In [116]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
135,35191277599329412,GALAXY,0.8607,KSD,2,NaN,0.8607,NaN,--,0.8607,GALAXY,3.0,2,0.8607,0.0,--|--,2,none
185,35191277599329412,GALAXY,0.8607,PNT,4,NaN,0.8607,GALAXY,--,0.8607,GALAXY,3.0,2,0.8607,0.0,--|--,2,none


In [117]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.7966
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift should be secure.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  9


In [118]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
145,35191277603523851,QSO,1.0445,KSD,3,NaN,1.0445,NaN,--,1.0445,QSO,3.5,1,1.0445,0.0,--|--,2,none
195,35191277603523851,QSO,1.0445,PNT,4,NaN,1.0445,GALAXY,--,1.0445,GALAXY,3.5,1,1.0445,0.0,--|--,2,none


In [119]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.6283
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. Redshift is robust. Bad calibration'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  10


In [120]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
109,35191281143513341,GALAXY,0.7841,KSD,2,NaN,0.7841,NaN,--,0.7841,GALAXY,3.0,2,0.7841,0.0,--|--,2,none
159,35191281143513341,GALAXY,0.7841,PNT,4,NaN,0.7841,GALAXY,--,0.7841,GALAXY,3.0,2,0.7841,0.0,--|--,2,none


In [121]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.1877
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'CaII and G band'
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  11


In [122]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
115,35191281147710346,GALAXY,1.1313,KSD,2,NaN,1.1313,NaN,--,1.1313,GALAXY,3.0,2,1.1313,0.0,--|Ca(K) missing from the model,2,none
165,35191281147710346,GALAXY,1.1313,PNT,4,NaN,1.1313,GALAXY,Ca(K) missing from the model,1.1313,GALAXY,3.0,2,1.1313,0.0,--|Ca(K) missing from the model,2,none


In [123]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.8376
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Possible broad [OII]. Be conservative'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  12


In [124]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
212,35191281160292265,GALAXY,0.6756,KSD,2,NaN,0.6756,NaN,--,0.6756,GALAXY,3.0,2,0.6756,0.0,--|--,2,none
262,35191281160292265,GALAXY,0.6756,NaN,4,NaN,0.6756,GALAXY,--,0.6756,GALAXY,3.0,2,0.6756,0.0,--|--,2,none


In [125]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Low S/N. Be conservative.'
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  13


In [126]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
235,35191281177068074,GALAXY,0.8527,KSD,2,NaN,0.8527,NaN,--,0.8527,GALAXY,3.0,2,0.8527,0.0,--|--,2,none
285,35191281177068074,GALAXY,0.8527,NaN,4,NaN,0.8527,GALAXY,--,0.8527,GALAXY,3.0,2,0.8527,0.0,--|--,2,none


In [127]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Possible CaII and  G band. Be conservative'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  14


In [128]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
247,35191281181264139,GALAXY,0.9746,KSD,3,NaN,0.7383,NaN,--,0.7383,GALAXY,2.0,2,0.9746,0.135937,--|Continuum is resolved But Very noisy signal...,2,none
297,35191281181264139,GALAXY,0.9746,NaN,1,NaN,0.9746,GALAXY,Continuum is resolved But Very noisy signal th...,0.9746,GALAXY,2.0,2,0.9746,0.135937,--|Continuum is resolved But Very noisy signal...,2,none


In [129]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.7383
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration drives the best fit'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  15


### and so on...

We should now recompute the conflicts, and not find any.

In [130]:
vi_conflict = VI.find_conflicts(vi_gp)

In [131]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

Let's check the number of conflicts again.

In [132]:
vi_conflict = VI.find_conflicts(vi_gp)

In [133]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [134]:
vi['vi_combined_flag'] = vi.groupby('TARGETID')['VI_quality'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TARGETID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [135]:
vi.keys()

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2', 'best redshift',
       'best spectype', 'vi_combined_flag', 'vi_diff', 'dz', 'vi_combined_z',
       'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [136]:
print(VI_dir+save_filename)

/global/homes/t/tlan/projects/VI_files/Andes_reinspection/LRG/truth_table_Andes_reinspection_LRG_68001_20200315_3.csv


In [137]:
vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',  
      'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER'].first().to_csv(VI_dir+save_filename)

<ipython-input-137-2063ed20beca>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',


Check that merged file reads in OK - check comments

In [138]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [139]:
merged_file.keys()

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'all VI comments',
       'merger comment', 'N_VI', 'DELTACHI2', 'FIBER'],
      dtype='object')

In [140]:
len(merged_file)

150

In [141]:
VI_dir

'/global/homes/t/tlan/projects/VI_files/Andes_reinspection/LRG/'